In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
from scipy.stats import ttest_ind
import math

%matplotlib inline

In [2]:
# some matplotlib params to make the defaults a bit more readable
mpl.rcParams['figure.figsize'] = [8.0, 6.0]
mpl.rcParams['font.size'] = 15
mpl.rcParams['axes.spines.right'] = False
mpl.rcParams['axes.spines.top'] = False

## Lagging and rolling weather features
This paper, titled [Predicting Culex pipiens/restuans population dynamics by interval lagged weather data](https://parasitesandvectors.biomedcentral.com/articles/10.1186/1756-3305-6-129#Tab1), published in the journal *Parasites and Vectors*, details the researchers efforts to model mosquito abundance in Chicago. They used a more extensive trap and weather dataset than us, allowing them to model mosquito population all the way back fro 1993. In this paper, they explain their methodology to exploring the best combination of lagged and rolled time features that best correlates with mosquito population. While our focus is not on forecasting mosquito population, but on classifying instances of West Nile Virus, we can still take inspiration from this paper on what some are some features worth lagging and rolling. 

As a starting point, we will follow their recommendations on how to use Tavg, AvgSpeed, and Daytime length, which we engineered as SunHours, as lagged and rolled features.

1. Tavg
    - The highest correlation with mosquito population was found from using a lag of 1 day and a rolling mean of 17 days.
2. AvgSpeed
    - The highest correlation was found from using a lag of 1 day and a rolling mean of 22 days.
3. SunHours
    - The highest correlation was found from using a lag of 28 days with a rolling mean of 11 days.
    - However, we do not have enough buffer to do this, as our weather data starts only about 4 weeks before the first trap is sampled for that year. As a compromise, we will use a lag of 24 days with a rolling mean of 4 days, in an effort to 'capture' temperature from roughly 4 weeks prior to the current time of trap sampling.

Additionally, the authors also found the greatest correlation for Humidity came from a lag of 23 days and a rolling mean of 83 days. We do not have nearly enough data for us to follow their findings, but they did also say that "a high relative humidity in the month prior the capture event had a positive effect on mosquito capture rates". As such, we will roll humidity over 28 days instead. 

Finally, the authors state that "the amount of precipitation during the previous generation had a stronger effect on the capture rates than the rain falling during the lifespan of the captured mosquitoes". The lifecycle of a mosquito is generally very temperature dependent, and thus it is difficult to pinpoint accurately the appropriate time lag between 2 successive generations. It can be anywhere between 2 weeks to a month. As a rough estimate for **total** precipitation over the lifetime of the previous generation, we will use a lag of 2 weeks coupled with a rolling **sum** of 2 weeks.

In [27]:
wx = pd.read_csv('./data_clean/weather_sam.csv')
wx.set_index('Date', inplace=True)
wx.head()


,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Sunrise,Sunset,CodeSum,PrecipTotal,...,SeaLevel,ResultSpeed,ResultDir,AvgSpeed,Heat/Cool,sunset,sunrise,SunHours,isRainy,has_Rain
Date,,,,,,,,,,,,,,,,,,,,,
2007-05-01,83,51.0,67.00,14.0,51,56.0,448,1849,,0.0,...,29.82,1.7,27,9.2,2.00,2007-05-01 18:49:00,2007-05-01 04:48:00,841.0,0,0
2007-05-02,59,42.5,50.75,-3.0,42,47.0,447,1850,BR,0.0,...,30.09,13.0,4,13.4,-14.25,2007-05-02 18:50:00,2007-05-02 04:47:00,843.0,0,0
2007-05-03,66,47.0,56.50,2.0,40,48.0,446,1851,,0.0,...,30.12,11.7,7,11.9,-8.50,2007-05-03 18:51:00,2007-05-03 04:46:00,845.0,0,0
2007-05-04,66,50.0,58.00,4.0,41,50.0,444,1852,RA,0.0,...,30.05,10.4,8,10.8,-7.00,2007-05-04 18:52:00,2007-05-04 04:44:00,848.0,1,0
2007-05-05,66,53.5,59.75,5.0,38,49.0,443,1853,,0.0,...,30.10,11.7,7,12.0,-5.25,2007-05-05 18:53:00,2007-05-05 04:43:00,850.0,0,0


In [16]:
wx.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1472 entries, 2007-05-01 to 2014-10-31
Data columns (total 24 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Tmax               1472 non-null   int64  
 1   Tmin               1472 non-null   float64
 2   Tavg               1472 non-null   float64
 3   Depart             1472 non-null   float64
 4   DewPoint           1472 non-null   int64  
 5   WetBulb            1472 non-null   float64
 6   Sunrise            1472 non-null   int64  
 7   Sunset             1472 non-null   int64  
 8   CodeSum            1472 non-null   object 
 9   PrecipTotal        1472 non-null   float64
 10  StnPressure        1469 non-null   float64
 11  SeaLevel           1467 non-null   float64
 12  ResultSpeed        1472 non-null   float64
 13  ResultDir          1472 non-null   int64  
 14  AvgSpeed           1472 non-null   float64
 15  Heat/Cool          1472 non-null   float64
 16  sunset        

In [30]:
def convert_to_celcius(fahrenheit):
    return round((fahrenheit-32) / 1.8, 3)

In [36]:
def rel_humidity_calc(dewpoint, temperature):
#     saturation_vapor_pressure = (6.11 * 10) ** (7.5 * temperature / (237.7 + temperature))
#     actual_vapor_pressure = (6.11 * 10) ** (7.5 * dewpoint / (237.7 + dewpoint))
#     rel_humidity = (actual_vapor_pressure / saturation_vapor_pressure)*100

    rel_humidity = 100*(np.exp((17.625*dewpoint)/(243.04+dewpoint))/np.exp((17.625*temperature)/(243.04+temperature)))
    return round(rel_humidity,2)

In [38]:
wx['Humidity'] = rel_humidity_calc(wx['DewPoint'].map(convert_to_celcius), wx['Tavg'].map(convert_to_celcius))

In [42]:
wx['Tavg_lag1_r18'] = wx['Tavg'].shift(1).rolling(17).mean()
wx['AvgSpeed_lag1_r22'] = wx['AvgSpeed'].shift(1).rolling(22).mean()
wx['SunHours_lag24_r4'] = wx['SunHours'].shift(24).rolling(4).mean()
wx['Humidity_r28'] = wx['Humidity'].rolling(28).mean()
wx['PrecipTotal'] = wx['PrecipTotal'].shift(14).rolling(14).sum()


In [44]:
wx.to_csv('./data_clean/weather.csv')

In [43]:
wx.head()

,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Sunrise,Sunset,CodeSum,PrecipTotal,...,sunrise,SunHours,isRainy,has_Rain,Humidity,Tavg_lag1_r18,AvgSpeed_lag1_r22,SunHours_lag25_r4,Humidity_r28,SunHours_lag24_r4
Date,,,,,,,,,,,,,,,,,,,,,
2007-05-01,83,51.0,67.00,14.0,51,56.0,448,1849,,NaN,...,2007-05-01 04:48:00,841.0,0,0,56.44,NaN,NaN,NaN,NaN,NaN
2007-05-02,59,42.5,50.75,-3.0,42,47.0,447,1850,BR,NaN,...,2007-05-02 04:47:00,843.0,0,0,71.86,NaN,NaN,NaN,NaN,NaN
2007-05-03,66,47.0,56.50,2.0,40,48.0,446,1851,,NaN,...,2007-05-03 04:46:00,845.0,0,0,53.89,NaN,NaN,NaN,NaN,NaN
2007-05-04,66,50.0,58.00,4.0,41,50.0,444,1852,RA,NaN,...,2007-05-04 04:44:00,848.0,1,0,53.08,NaN,NaN,NaN,NaN,NaN
2007-05-05,66,53.5,59.75,5.0,38,49.0,443,1853,,NaN,...,2007-05-05 04:43:00,850.0,0,0,44.36,NaN,NaN,NaN,NaN,NaN
